In [57]:
import json

ref_mobility = "sgp4"
new_mobility = "kepler"
ref_mobility_periods_path = "/workspaces/ma-max-wendler/examples/space_veins/stats/comm_periods/iridiumNEXT/sgp4/leoIRIDIUM[106]_communication-periods.json"
new_mobility_periods_path = "/workspaces/ma-max-wendler/examples/space_veins/stats/comm_periods/iridiumNEXT/kepler/leoIRIDIUM[106]_communication-periods.json"
comm_period_groups_path = "/workspaces/ma-max-wendler/examples/space_veins/stats/compare_comm_periods/iridiumNEXT/sgp4-kepler_leoIRIDIUM[100]_communication_comparison.json"

### load communication period statistics ###
ref_mobility_stats = None
with open(ref_mobility_periods_path, "r") as in_json:
    ref_mobility_stats = json.load(in_json)
new_mobility = new_mobility
new_mobility_stats = None
with open(new_mobility_periods_path, "r") as in_json:
    new_mobility_stats = json.load(in_json)

# load communication period comparison
comm_periods = {}
with open(comm_period_groups_path, "r") as in_json:
    comm_periods = json.load(in_json)

period_groups = comm_periods["period_groups"]

In [58]:
p_group = period_groups[0]
ref_angles = ref_mobility_stats["angles"][ p_group[ref_mobility]["period_idx"] ]
new_angles = new_mobility_stats["angles"][ p_group[new_mobility]["period_idx"] ]
p_group[ref_mobility]["period_idx"]

0

In [59]:
len(ref_angles), len(new_angles)

(323, 310)

In [60]:
# create test list to be interpolated where only every second value between start and end of list is used
# for this, an odd number of entries is required (1st,3rd,5th,7th,9th,...)
ref_angles_test = [ref_angles[0]]
for i in range(2, len(ref_angles), 2):
    ref_angles_test.append(ref_angles[i])
len(ref_angles_test)

162

In [61]:
import numpy as np
from scipy.interpolate import interp1d

# interpolate test list
ref_angles_test_x = np.linspace(0, 1, len(ref_angles_test))
ref_angles_x = np.linspace(0, 1, len(ref_angles))

interpolators = {}
interpolators["linear"] = interp1d(ref_angles_test_x, ref_angles_test, kind="linear")
interpolators["zero"] = interp1d(ref_angles_test_x, ref_angles_test, kind="zero")
interpolators["slinear"] = interp1d(ref_angles_test_x, ref_angles_test, kind="slinear")
interpolators["quadratic"] = interp1d(ref_angles_test_x, ref_angles_test, kind="quadratic")
interpolators["cubic"] = interp1d(ref_angles_test_x, ref_angles_test, kind="cubic")

ref_angles_test_interps = {}
for interp_kind_function_tuple in interpolators.items():
    ref_angles_test_interps[interp_kind_function_tuple[0]] = list( interp_kind_function_tuple[1](ref_angles_x) )

In [62]:
from math import sqrt

root_mean_square_errors = {}
for interp_kind_vals_tuple in ref_angles_test_interps.items():
    
    vals = interp_kind_vals_tuple[1]
    
    square_error_sum = 0
    for i in range(len(ref_angles)):
        square_error_sum += (ref_angles[i] - vals[i]) * (ref_angles[i] - vals[i])
    
    mean_square_error = square_error_sum / len(ref_angles)
    root_mean_square_errors[interp_kind_vals_tuple[0]] = sqrt(mean_square_error) 

root_mean_square_errors

{'linear': 0.0005832266129306626,
 'zero': 0.09922449535352873,
 'slinear': 0.0005832266129315459,
 'quadratic': 7.143213553146479e-08,
 'cubic': 2.4209948622463036e-08}

In [63]:
min(root_mean_square_errors.values())

2.4209948622463036e-08

=> here, cubic interpolation is best